In [1]:
import findspark 
findspark.init()
findspark.find()
from pyspark.sql import SparkSession
import pandas as pd
from datetime import datetime, timedelta,date
import pyspark.sql.functions as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import datetime as dt
import xlsxwriter
from sklearn.linear_model import LinearRegression 
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType, FloatType ,DateType
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans

In [2]:
spark = SparkSession.builder.config('spark.executor.memory','4G').appName('recommend').config('spark.driver.cores','4') \
.config('spark.driver.memory','4G') \
.config('spark.driver.maxResultSize','4G').getOrCreate()
df=spark.read.csv('C:/Users/PC/Downloads/stores.csv',header=True,inferSchema=True)

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [4]:
columns=['Store_Area',
 'Items_Available',
 'Daily_Customer_Count',
 'Store_Sales']

In [5]:
assemler=VectorAssembler(inputCols=columns,outputCol='features')

In [6]:
df=assemler.transform(df)
scaler = StandardScaler(inputCol='features',outputCol='scalerfeatures')
scaler_model=scaler.fit(df)
df=scaler_model.transform(df)
df.printSchema()

root
 |-- Store ID : integer (nullable = true)
 |-- Store_Area: integer (nullable = true)
 |-- Items_Available: integer (nullable = true)
 |-- Daily_Customer_Count: integer (nullable = true)
 |-- Store_Sales: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- scalerfeatures: vector (nullable = true)



In [24]:
kmeans=KMeans(featuresCol='scalerfeatures',k=4)
model_df=kmeans.fit(df)
model_df.transform(df).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  218|
|         3|  212|
|         2|  204|
|         0|  262|
+----------+-----+

